In [9]:
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import time

import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import os

# CONFIGURA TUS CREDENCIALES Y URL DE OKTA
OKTA_USERNAME = "yonatan.osorio@openenglish.com"
OKTA_PASSWORD = "Matias2023#"
OKTA_URL = "https://openeducation.okta.com"
reporte_casos_url = "https://openeducation.my.salesforce.com/00O3r000006mGPy"

ruta_excel_salida = "G:\\Unidades compartidas\\Workforce\\r\\reporte.xlsx"

carpeta_descargas = "G:\\Unidades compartidas\\Workforce\\r\\descargas"

scope = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("C:\\Users\\yonat\\OneDrive\\Documentos\\programas\\auto.json", scope)
client = gspread.authorize(creds)

# Configurar opciones de Chrome
options = Options()
options.add_experimental_option("prefs", {
    "download.default_directory": carpeta_descargas,  # sin comillas dobles
    "download.prompt_for_download": False,
    "directory_upgrade": True
})

# ABRIR CHROME
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get(OKTA_URL)

# ESPERA QUE CARGUE LA PÁGINA DE OKTA
try:
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.ID, "input28"))
    )
    print("✅ Página de Okta cargada correctamente.")
except TimeoutException:
    print("❌ Tiempo de espera agotado. No se cargó la página de Okta.")
    driver.quit()
    exit()

# COMPLETAR LOGIN EN OKTA
try:
    driver.find_element(By.ID, "input28").send_keys(OKTA_USERNAME)
    driver.find_element(By.ID, "input36").send_keys(OKTA_PASSWORD)
    driver.find_element(By.XPATH, '//input[@value="Iniciar sesión"]').click()
    print("🔐 Intentando iniciar sesión...")
except NoSuchElementException:
    print("❌ No se encontró el formulario de login.")
    driver.quit()
    exit()

# ESPERA A QUE CARGUE EL DASHBOARD Y CLIC EN SALESFORCE
try:
    salesforce_btn = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, '//a[contains(@href, "salesforce")]'))
    )
    salesforce_btn.click()
    print("🚀 Acceso a Salesforce exitoso.")
    print("✅ Ya estás dentro de Salesforce.")
    # Ir al link del reporte directamente
    
    #reporte de casos   
    print("🔗 Ingreso a informes de Casos...")
    time.sleep(3)
    driver.execute_script("window.open('https://openeducation.my.salesforce.com/00O3r000006mGPy', '_blank');")
    driver.switch_to.window(driver.window_handles[1])

    ayer = datetime.now() - timedelta(days=1)
    fecha_ayer = ayer.strftime('%d/%m/%Y')  # Ajusta el formato si Salesforce espera otro
    print("📅 Fecha del día anterior:", fecha_ayer)
    time.sleep(10)
    date_picker = driver.find_element(By.XPATH, '//*[@id="edate"]')
    date_picker.click()
    date_picker.clear()

    time.sleep(1)
    date_picker.send_keys(fecha_ayer)
    
    driver.find_element(By.XPATH, '//input[@value="Ejecutar informe"]').click()
    time.sleep(2)
    driver.find_element(By.XPATH, '//input[@value="Exportar detalles"]').click()
    time.sleep(3)
    dropdown = Select(driver.find_element(By.ID, "xf"))
    dropdown.select_by_value("localecsv")
    time.sleep(1)
    driver.find_element(By.XPATH, '//input[@value="Exportar"]').click()

    #Copiar reporte de casos
    def archivo_mas_reciente(carpeta, extension=".csv"):
        archivos = [f for f in os.listdir(carpeta) if f.endswith(extension)]
        if not archivos:
            raise FileNotFoundError("No se encontró ningún archivo CSV en la carpeta.")
        
        archivos = [os.path.join(carpeta, f) for f in archivos]
        archivo_reciente = max(archivos, key=os.path.getctime)
        return archivo_reciente
    ruta_archivo = archivo_mas_reciente(r"G:\\Unidades compartidas\\Workforce\\r\\descargas", extension=".csv")  # o ".xls"

    print("📄 Archivo más reciente:", ruta_archivo)
    df = pd.read_csv(ruta_archivo, sep=None, engine='python')
    df = df[:-5]

    # Limpiar la hoja anterior (opcional)
    with pd.ExcelWriter(ruta_excel_salida, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name="Casos", index=False)

    print("✅ Hoja 'Casos' actualizada en Google Sheets.")
    time.sleep(15)
    
#-----------------------------------------------------------------------------------------------------------------------------#
    #reporte de Ohs   
    print("🔗 Ingreso a informes de Ohs...")
    time.sleep(3)
    driver.execute_script("window.open('https://openeducation.my.salesforce.com/00O0Z0000070nf9','_parent');")
    driver.switch_to.window(driver.window_handles[1])
    driver.find_element(By.XPATH, '//input[@value="Exportar detalles"]').click()
    time.sleep(3)
    dropdown = Select(driver.find_element(By.ID, "xf"))
    dropdown.select_by_value("localecsv")
    time.sleep(1)
    driver.find_element(By.XPATH, '//input[@value="Exportar"]').click()
        
    #Copiar reporte de Ohs
    def archivo_mas_reciente(carpeta, extension=".csv"):
        archivos = [f for f in os.listdir(carpeta) if f.endswith(extension)]
        if not archivos:
            raise FileNotFoundError("No se encontró ningún archivo CSV en la carpeta.")
        
        archivos = [os.path.join(carpeta, f) for f in archivos]
        archivo_reciente = max(archivos, key=os.path.getctime)
        return archivo_reciente
    ruta_archivo = archivo_mas_reciente(r"G:\\Unidades compartidas\\Workforce\\r\\descargas", extension=".csv")  # o ".xls"

    print("📄 Archivo más reciente:", ruta_archivo)
    df = pd.read_csv(ruta_archivo, sep=None, engine='python')
    df = df[:-5]

    # Limpiar la hoja anterior (opcional)
    with pd.ExcelWriter(ruta_excel_salida, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name="Ohs", index=False)

    print("✅ Hoja 'Ohs' actualizada en Google Sheets.")
    time.sleep(15)

#---------------------------------------------------------------------------------------------------------------------------------#
    #reporte de OH_QuienProcesa
    print("🔗 Ingreso a informes de OH_QuienProcesa...")
    time.sleep(3)
    driver.execute_script("window.open('https://openeducation.my.salesforce.com/00O0Z0000070lP4', '_parent');")
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(2)
    driver.find_element(By.XPATH, '//input[@value="Exportar detalles"]').click()
    time.sleep(3)
    dropdown = Select(driver.find_element(By.ID, "xf"))
    dropdown.select_by_value("localecsv")
    time.sleep(1)
    driver.find_element(By.XPATH, '//input[@value="Exportar"]').click()
        
    #Copiar reporte de OH_QuienProcesa
    def archivo_mas_reciente(carpeta, extension=".csv"):
        archivos = [f for f in os.listdir(carpeta) if f.endswith(extension)]
        if not archivos:
            raise FileNotFoundError("No se encontró ningún archivo CSV en la carpeta.")
        
        archivos = [os.path.join(carpeta, f) for f in archivos]
        archivo_reciente = max(archivos, key=os.path.getctime)
        return archivo_reciente
    ruta_archivo = archivo_mas_reciente(r"G:\\Unidades compartidas\\Workforce\\r\\descargas", extension=".csv")  # o ".xls"

    print("📄 Archivo más reciente:", ruta_archivo)
    df = pd.read_csv(ruta_archivo, sep=None, engine='python')
    df = df[:-5]

    # Limpiar la hoja anterior (opcional)
    with pd.ExcelWriter(ruta_excel_salida, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name="OH_QuienProcesa", index=False)

    print("✅ Hoja 'OH_QuienProcesa' actualizada en Google Sheets.")
    time.sleep(15)
    
#---------------------------------------------------------------------------------------------------------------------------------#

    #reporte de Acc&Subs&OH   
    print("🔗 Ingreso a informes de Acc&Subs&OH...")
    time.sleep(3)
    driver.execute_script("window.open('https://openeducation.my.salesforce.com/00O3r000006vjiK','_parent');")
    driver.switch_to.window(driver.window_handles[1])
    driver.find_element(By.XPATH, '//input[@value="Exportar detalles"]').click()
    time.sleep(3)
    dropdown = Select(driver.find_element(By.ID, "xf"))
    dropdown.select_by_value("localecsv")
    time.sleep(1)
    driver.find_element(By.XPATH, '//input[@value="Exportar"]').click()
        
    #Copiar reporte de Acc&Subs&OH
    def archivo_mas_reciente(carpeta, extension=".csv"):
        archivos = [f for f in os.listdir(carpeta) if f.endswith(extension)]
        if not archivos:
            raise FileNotFoundError("No se encontró ningún archivo CSV en la carpeta.")
        
        archivos = [os.path.join(carpeta, f) for f in archivos]
        archivo_reciente = max(archivos, key=os.path.getctime)
        return archivo_reciente
    ruta_archivo = archivo_mas_reciente(r"G:\\Unidades compartidas\\Workforce\\r\\descargas", extension=".csv")  # o ".xls"

    print("📄 Archivo más reciente:", ruta_archivo)
    df = pd.read_csv(ruta_archivo, sep=None, engine='python')
    df = df[:-5]

    # Limpiar la hoja anterior (opcional)
    with pd.ExcelWriter(ruta_excel_salida, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name="Acc&Subs&OH", index=False)

    print("✅ Hoja 'Acc&Subs&OH' actualizada en Google Sheets.")
    time.sleep(15)

#----------------------------------------------------------------------------------------------------------------------------------#

    #reporte de transfer
    spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/1WaNSjdgPVJh0MaAqUxOxoO5Lu1WK9DQhhvsJQvQSJrg/edit?gid=0#gid=0")  # o client.open_by_url("URL_DEL_SHEET")

        # Selecciona la hoja (worksheet)
    worksheet = spreadsheet.worksheet("hoja 1")

        # Obtiene todos los datos
    data = worksheet.get_all_values()

    df = pd.DataFrame(data[1:], columns=data[0])  # Usa la primera fila como encabezado
    # Guarda como Excel
    df.to_excel("transfer.xlsx", index=False)
    
    #Copiar reporte de Transfer
    def archivo_mas_reciente(carpeta, extension=".csv"):
        archivos = [f for f in os.listdir(carpeta) if f.endswith(extension)]
        if not archivos:
            raise FileNotFoundError("No se encontró ningún archivo CSV en la carpeta.")
        
        archivos = [os.path.join(carpeta, f) for f in archivos]
        archivo_reciente = max(archivos, key=os.path.getctime)
        return archivo_reciente
    ruta_archivo = archivo_mas_reciente(r"G:\\Unidades compartidas\\Workforce\\r\\descargas", extension=".csv")  # o ".xls"

    print("📄 Archivo más reciente:", ruta_archivo)
    df = pd.read_csv(ruta_archivo, sep=None, engine='python')
    df = df[:-5]

    # Limpiar la hoja anterior (opcional)
    with pd.ExcelWriter(ruta_excel_salida, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name="Transfer", index=False)

    print("✅ Hoja 'Transfer' actualizada en Google Sheets.")
    

except TimeoutException:
    print("❌ No se encontró el botón de Salesforce.")
    driver.quit()
    exit()



# Puedes dejar el navegador abierto o cerrarlo después de pruebas:
#driver.quit()


✅ Página de Okta cargada correctamente.
🔐 Intentando iniciar sesión...
🚀 Acceso a Salesforce exitoso.
✅ Ya estás dentro de Salesforce.
🔗 Ingreso a informes de Casos...
📅 Fecha del día anterior: 14/04/2025
📄 Archivo más reciente: G:\\Unidades compartidas\\Workforce\\r\\descargas\report1744768642986.csv
✅ Hoja 'Casos' actualizada en Google Sheets.
🔗 Ingreso a informes de Ohs...
📄 Archivo más reciente: G:\\Unidades compartidas\\Workforce\\r\\descargas\report1744768746726.csv
✅ Hoja 'Ohs' actualizada en Google Sheets.
🔗 Ingreso a informes de OH_QuienProcesa...
📄 Archivo más reciente: G:\\Unidades compartidas\\Workforce\\r\\descargas\report1744768771503.csv
✅ Hoja 'OH_QuienProcesa' actualizada en Google Sheets.
🔗 Ingreso a informes de Acc&Subs&OH...
📄 Archivo más reciente: G:\\Unidades compartidas\\Workforce\\r\\descargas\report1744768804322.csv
✅ Hoja 'Acc&Subs&OH' actualizada en Google Sheets.
📄 Archivo más reciente: G:\\Unidades compartidas\\Workforce\\r\\descargas\report1744768831754.cs

In [6]:
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import os

# Autenticación
scope = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("C:\\Users\\yonat\\OneDrive\\Documentos\\programas\\auto.json", scope)
client = gspread.authorize(creds)

# Abrir el archivo de Sheets por nombre o ID
spreadsheet = client.open_by_key("1FDok-e6IZdtTfoPbCou2St7fmiuw4eOKhzkd4unsHlc")

# Seleccionar la hoja a actualizar
hoja = spreadsheet.worksheet("ohs")

def archivo_mas_reciente(carpeta, extension=".csv"):
    archivos = [f for f in os.listdir(carpeta) if f.endswith(extension)]
    if not archivos:
        raise FileNotFoundError("No se encontró ningún archivo CSV en la carpeta.")
    
    archivos = [os.path.join(carpeta, f) for f in archivos]
    archivo_reciente = max(archivos, key=os.path.getctime)
    return archivo_reciente

# Ejemplo de uso:
ruta_archivo = archivo_mas_reciente("C:\\Users\\yonat\\OneDrive\\Documentos\\programas\\reporte")
print("📄 Archivo más reciente:", ruta_archivo)

# Leer los datos desde el CSV
df = pd.read_csv(ruta_archivo, sep=None, engine='python')
df = df[:-5]



# Limpiar la hoja anterior (opcional)
hoja.clear()

# Escribir los nuevos datos
set_with_dataframe(hoja, df)

print("✅ Hoja 'Casos' actualizada en Google Sheets.")



📄 Archivo más reciente: C:\Users\yonat\OneDrive\Documentos\programas\reporte\report1744573219992.csv
✅ Hoja 'Casos' actualizada en Google Sheets.


In [4]:
import os

def archivo_mas_reciente(carpeta, extension=".csv"):
    archivos = [f for f in os.listdir(carpeta) if f.endswith(extension)]
    if not archivos:
        raise FileNotFoundError("No se encontró ningún archivo CSV en la carpeta.")
    
    archivos = [os.path.join(carpeta, f) for f in archivos]
    archivo_reciente = max(archivos, key=os.path.getctime)
    return archivo_reciente

# Ejemplo de uso:
ruta_archivo = archivo_mas_reciente("C:\\Users\\yonat\\OneDrive\\Documentos\\programas\\reporte")
print("📄 Archivo más reciente:", ruta_archivo)

📄 Archivo más reciente: C:\Users\yonat\OneDrive\Documentos\programas\reporte\gf.csv


In [39]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Autenticación con Google API usando las credenciales
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("C:\\Users\\yonat\\OneDrive\\Documentos\\programas\\auto.json", scope)
client = gspread.authorize(creds)

# Abrir el archivo de Google Sheets por nombre o URL
sheet = client.open_by_key("1WaNSjdgPVJh0MaAqUxOxoO5Lu1WK9DQhhvsJQvQSJrg").worksheet("hoja 1")

# Leer todos los datos de la hoja
data = sheet.get_all_values()

# Mostrar los datos (opcional)
for fila in data:
    print(fila)

# Si quieres acceder a una celda específica
valor = sheet.cell(1, 1).value  # Obtiene el valor de la celda A1
print(valor)


['Asesor transfer:', 'Supervisor', 'Correo electrónico', 'Chanel Slack', '# Caso', 'Detalles', 'Recuperado', 'Producto Recuperado', 'Cantidad Recuperada', 'Enviado por', 'Fecha', 'Hora Flujo', 'Country', 'Falida S/N', 'Suscripcion', 'Organizacion', 'Flujo de Transfer', 'Propietario  Caso', 'Estate Case', 'Country Case', 'Organization Case', 'Type Case', 'Propietario KO', 'Country KO', 'Suscripcion KO', 'Type Case KO', 'Check D.S.', 'KO Direct Sale', 'Subsc Direct Sale', 'Caso Reasignado', 'Ultima Actualizacion (Lead & Ag OE)']
['Julian Pedraza', 'Jhonatan Buitrago', 'ccesarm21@gmail.com', '#recovery-oe-adulto', '20739998', 'Cliente corta en transfer, se marca y no contesta', ':x: No - Cuelga', '', '', 'Mivia Garcia', '2025-04-01', '1 Apr 2025 7:35:04', 'United State', 'Fallida', '', 'Open Adulto', '1', 'Mivia Garcia', 'Abierto', 'United State', 'Open English', 'Failed Sale', '0', '0', '0', '0', '0', '', '', '', '']
['Barbara Sosa', 'Andres Garcia', 'andreahersan1@gmail.com', '#recovery

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import pandas as pd
import os
# -----------------------
# 1. Autenticación con Google Drive
# -----------------------
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("C:\\Users\\yonat\\OneDrive\\Documentos\\programas\\auto.json", scope)
client = gspread.authorize(creds)

# -----------------------
# 2. Descargar el archivo desde Google Drive
# -----------------------
# ID del archivo en Google Drive (puedes copiarlo desde la URL del archivo)
file_id = "1WaNSjdgPVJh0MaAqUxOxoO5Lu1WK9DQhhvsJQvQSJrg"  # reemplaza esto

# Define el nombre temporal de descarga
archivo_descargado = "C:\\Users\\yonat\\OneDrive\\Documentos\\programas\\reporte\\transfer.xlsx"

# Descargar el archivo
archivo = drive.CreateFile({'id': file_id})
archivo.GetContentFile(archivo_descargado)

print("✅ Archivo descargado exitosamente.")

# -----------------------
# 3. Leer el archivo descargado (asumimos que es un Excel)
# -----------------------
df = pd.read_excel(archivo_descargado)

# -----------------------
# 4. Guardar los datos en un archivo Excel personalizado
# -----------------------
nombre_archivo_salida = "transfer.xlsx"
ruta_salida = os.path.join(os.getcwd(), nombre_archivo_salida)
nombre_hoja = "hoja 1"

# Guardar DataFrame en archivo nuevo con hoja personalizada
df.to_excel(ruta_salida, sheet_name=nombre_hoja, index=False)

print(f"✅ Archivo guardado como: {ruta_salida}")

# -----------------------
# 5. (Opcional) Eliminar archivo temporal
# -----------------------


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import pandas as pd
import os

# -----------------------
# 1. Autenticación con Google Drive
# -----------------------
gauth = GoogleAuth()
gauth.LocalWebserverAuth()  # Abre navegador para iniciar sesión
drive = GoogleDrive(gauth)  # ← ESTA LÍNEA ES LA QUE TE FALTABA

# -----------------------
# 2. Descargar archivo desde Google Drive
# -----------------------
file_id = "1WaNSjdgPVJh0MaAqUxOxoO5Lu1WK9DQhhvsJQvQSJrg"  # Reemplaza con tu ID real
archivo_descargado = "C:\\Users\\yonat\\OneDrive\\Documentos\\programas\\reporte\\transfer.xlsx"

# Descargar el archivo usando el ID
archivo = drive.CreateFile({'id': file_id})
archivo.GetContentFile(archivo_descargado)

print("✅ Archivo descargado exitosamente en:", archivo_descargado)

# -----------------------
# 3. Leer archivo y guardarlo como personalizado
# -----------------------
df = pd.read_excel(archivo_descargado)

# Guardar como nuevo archivo Excel con hoja personalizada
salida = "C:\\Users\\yonat\\OneDrive\\Documentos\\programas\\reporte\\mi_reporte_personalizado.xlsx"
df.to_excel(salida, sheet_name="DatosImportados", index=False)

print("✅ Reporte personalizado guardado en:", salida)



In [48]:
import gdown

# ID del archivo de Google Drive
file_id = '1WaNSjdgPVJh0MaAqUxOxoO5Lu1WK9DQhhvsJQvQSJrg'

# Crear la URL de Google Drive para descargar el archivo
url = f'https://drive.google.com/uc?export=download&id={file_id}'

# Descargar el archivo a la ubicación deseada
output = 'archivo_descargado.xlsx'  # Cambia el nombre del archivo según lo necesites
gdown.download(url, output, quiet=False)

ModuleNotFoundError: No module named 'gdown'